In [1]:
max_vector_len = 968 

In [5]:
import os
import subprocess
import csv
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(125)

def extract(v):
    return v.data.storage().tolist()

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight.data, gain=1)
        
class Generator(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map4 = nn.Linear(hidden_size, output_size)
        self.f = f 
 
    def forward(self, x):
        x = self.f(self.map1(x))
        return self.f(self.map4(x))

class Discriminator(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f 

    def forward(self, x):
        x = self.f(self.map1(x))
        return self.map3(x)

def load_data(filename):
    x = []
    with open(filename,'r') as data:
        lines = data.readlines()
        for line in lines:
            line=line.strip('\n')
            x.append(list(line))
    x = np.array(x,dtype=np.float64)
    return x

# Model parameters
feature_dims = max_vector_len
z_dims = 20
g_input_size = feature_dims + z_dims
g_output_size = feature_dims
g_hidden_size = int(math.sqrt(g_input_size+g_output_size)+10)
d_input_size = g_output_size
d_output_size = 1
d_hidden_size = int(math.sqrt(d_input_size+d_output_size)+10)

# 100*32*5=16000<30000*0.8=24000
epochs = 100 
batch_size = 32
print_interval = 10

d_learning_rate = 5e-5
g_learning_rate = 5e-5

clip = [-0.01, 0.01] 

d_steps = 5 # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
g_steps = 1 

discriminator_activation_function = torch.sigmoid
generator_activation_function = torch.sigmoid

benign = torch.zeros([batch_size,1]).reshape(-1)
mal = torch.ones([batch_size,1]).reshape(-1)

G = Generator(input_size=g_input_size,
              hidden_size=g_hidden_size,
              output_size=g_output_size,
              f=generator_activation_function)
G.apply(init_weights)
D = Discriminator(input_size=d_input_size,
                  hidden_size=d_hidden_size,
                  output_size=d_output_size,
                  f=discriminator_activation_function)  
D.apply(init_weights)
d_optimizer = optim.RMSprop(D.parameters(), lr=d_learning_rate)
g_optimizer = optim.RMSprop(G.parameters(), lr=g_learning_rate)

# Load the dataset
xmal = load_data('Mirai_vector_mal.txt')
ymal = np.ones((xmal.shape[0], 1), dtype=np.int)
xben = load_data('Mirai_vector_ben.txt')
yben = np.zeros((xben.shape[0], 1), dtype=np.int)
# Malicious packets are divided into training set and test set
train_size = int(xmal.shape[0] * 0.8)
test_size = xmal.shape[0] - train_size 
xtrain_mal = xmal[0:train_size, :]
xtest_mal = xmal[train_size:, :]
ytrain_mal = ymal[0:train_size, :]
ytest_mal = ymal[train_size:, :]
# The benign packets are divided into training set and test set, the number of training sets is the same as the training set of malicious packets
xtrain_ben = xben[0:train_size, :]
xtest_ben = xben[train_size:, :]
ytrain_ben = yben[0:train_size, :]
ytest_ben = yben[train_size:, :]

xtrain_mal = torch.Tensor(xtrain_mal)
xtest_mal = torch.Tensor(xtest_mal)
xtrain_ben = torch.Tensor(xtrain_ben)
xtest_ben = torch.Tensor(xtest_ben)

index_of_data_D_ben = 0     
index_of_data_D_mal = 0    
index_of_data_G = 0

for epoch in range(1, epochs+1): 
    for d_index in range(d_steps):
        D.zero_grad() # Zero gradient
        
        #  1A: Train D on real
        idx = np.array(range(index_of_data_D_ben, index_of_data_D_ben+batch_size))
        index_of_data_D_ben += batch_size
        xben_batch = xtrain_ben[idx]
        d_real_data = Variable(xben_batch)
        d_real_decision = D(d_real_data)
        d_real_error = torch.mean(d_real_decision.reshape(-1))
        d_real_error.backward(benign.mean())
        
        #  1B: Train D on fake
        idx = np.array(range(index_of_data_D_mal, index_of_data_D_mal+batch_size))
        index_of_data_D_mal += batch_size
        xmal_batch = xtrain_mal[idx]
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (batch_size, z_dims))
        d_gen_input = Variable(torch.cat((xmal_batch, torch.Tensor(noise)), 1))
        d_fake_data = G(d_gen_input).detach() # detach to avoid training G on these labels
        d_fake_decision = D(d_fake_data)
        d_fake_error = torch.mean(d_fake_decision.reshape(-1))
        d_fake_error.backward(mal.mean())
        
        d_optimizer.step() # Only optimizes D's parameters; changes based on stored gradients from backward()
        
        # Clip weights of discriminator
        for p in D.parameters():
            p.data.clamp_(clip[0], clip[1])
            
        dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]
    
    for g_index in range(g_steps):
        G.zero_grad()
        
        idx = np.array(range(index_of_data_G, index_of_data_G+batch_size))
        index_of_data_G += batch_size
        xmal_batch = xtrain_mal[idx]
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (batch_size, z_dims))
        gen_input = Variable(torch.cat((xmal_batch, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(gen_examples)
        g_error = torch.mean(dg_fake_decision.reshape(-1))
        g_error.backward(benign.mean())
        g_loss = extract(g_error)[0]
        
        g_optimizer.step()  # Only optimizes G's parameters

    if epoch % print_interval == 0:
        # D(benign)
        d_real_data = Variable(xtest_ben)
        d_real_decision = D(d_real_data)
        d_real_error = torch.mean(d_real_decision.reshape(-1))
        d_loss = extract(d_real_error)[0]
        #print("len of d_loss: %d" % (len(d_loss)))
        print("d_loss: %f" % (d_loss))
        # D(G(benign))
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (xben.shape[0] - train_size, z_dims))
        gen_input = Variable(torch.cat((xtest_ben, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(Variable(gen_examples))
        dg_error = torch.mean(dg_fake_decision.reshape(-1))
        dg_loss = extract(dg_error)[0]
        #print("len of dg_loss: %f" % (len(dg_loss)))
        print("dg_loss: %f" % (dg_loss))
        # Threshold
#         threshold = 0
#         for i in range(0, len(d_loss)):
#             threshold = max(threshold, abs(d_loss[i]-dg_loss[i]))
        threshold = abs(d_loss-dg_loss)
        print("threshold: %f" % (threshold)) 
        
        
#         # D(Malicious)
#         d_real_data = Variable(xtest_mal)
#         d_real_decision = D(d_real_data)
#         d_real_error = d_real_decision.reshape(-1)
#         d_loss = extract(d_real_error)
#         print("len of d_loss: %d" % (len(d_loss)))
        # D(G(Malicious))
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (xmal.shape[0] - train_size, z_dims))
        gen_input = Variable(torch.cat((xtest_mal, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(Variable(gen_examples))
        dg_error = dg_fake_decision.reshape(-1)
        dg_loss = extract(dg_error)
        print("len of dg_loss: %f" % (len(dg_loss)))
        
        # 计算 Precision Recall F-score
        positive_sum = 6001
        positive_num = 0
        FP = 0
        label_list = []
        for i in range(0, len(dg_loss)):
            if abs(d_loss-dg_loss[i]) > threshold:
                label_list.append('1')
                if ytest_mal[i][0] == 0:
                    FP += 1
                elif ytest_mal[i][0] == 1:
                    positive_num += 1
            else:
                label_list.append('0')
        

        if positive_num == 0:
            Precision = 0
            Recall = 0
            F_score = 0
        else:
            Precision = positive_num / (positive_num + FP)
            Recall = positive_num / positive_sum
            F_score = 2 * Recall * Precision / (Recall + Precision)
        print('Precision: {0:0.4f}'.format(Precision)) 
        print('Recall: {0:0.4f}'.format(Recall)) 
        print('F-score: {0:0.4f}'.format(F_score)) 
        print("%d [D real_err: %f, fake_err: %f] [G loss: %f]" % (epoch, dre, dfe, g_loss))          


d_loss: -0.089181
dg_loss: -0.116909
threshold: 0.027728
len of dg_loss: 6001.000000
Precision: 1.0000
Recall: 0.9055
F-score: 0.9504
10 [D real_err: -0.088375, fake_err: -0.115857] [G loss: -0.116930]
d_loss: -0.123233
dg_loss: -0.161909
threshold: 0.038676
len of dg_loss: 6001.000000
Precision: 1.0000
Recall: 0.9125
F-score: 0.9543
20 [D real_err: -0.122109, fake_err: -0.161197] [G loss: -0.161906]
d_loss: -0.152877
dg_loss: -0.197812
threshold: 0.044935
len of dg_loss: 6001.000000
Precision: 1.0000
Recall: 0.9192
F-score: 0.9579
30 [D real_err: -0.152122, fake_err: -0.197132] [G loss: -0.197815]
d_loss: -0.187412
dg_loss: -0.245279
threshold: 0.057867
len of dg_loss: 6001.000000
Precision: 1.0000
Recall: 0.9252
F-score: 0.9611
40 [D real_err: -0.186451, fake_err: -0.244080] [G loss: -0.245328]
d_loss: -0.231224
dg_loss: -0.317487
threshold: 0.086263
len of dg_loss: 6001.000000
Precision: 1.0000
Recall: 0.9257
F-score: 0.9614
50 [D real_err: -0.231154, fake_err: -0.315947] [G loss: -

In [6]:
print(len(ytest_mal))

6001


In [ ]:
# No random seed
Precision: 1.0000
Recall: 0.9183
F-score: 0.9574
30 [D real_err: -0.144911, fake_err: -0.190961] [G loss: -0.191560]
#
torch.manual_seed(200)
Precision: 1.0000
Recall: 0.8794
F-score: 0.9358
10 [D real_err: -0.053383, fake_err: -0.081003] [G loss: -0.082181]
# *
torch.manual_seed(125)
Precision: 1.0000
Recall: 0.9257
F-score: 0.9614
50 [D real_err: -0.231154, fake_err: -0.315947] [G loss: -0.317548]